In [1]:
from memory_reveal_cards_game import MemoryGame
from minimum_vertex_cover_vertices import MinimumVertexCoverVertices
from logChoice import LogChoice
from bigChoice import BigChoice
from fixedParameters import FixedParameters
from pso import pso_solve
import numpy as np
import random
from utils import smallest_dominating_set, greedy_dominating_set, generate_random_graph

In [3]:
choice = LogChoice()
choiceParameters = FixedParameters()
problem = MemoryGame(choice, choiceParameters, 0.01, 36, 50)

In [4]:
init_positions = lambda num_particles: [np.random.random((1, 2)) * 2 - 1 for i in range(num_particles)]
init_velocities = lambda num_particles: [np.zeros((1, 2)) for i in range(num_particles)]   

def fitness(p):
    total = 0
    count = 10
    ol = 0

    choiceParameters.set(p[0][0], p[0][1])

    for i in range(count):
        cards = problem.random_cards()
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        return ol + 2 * (total / (count - ol)) / problem.limit
    else:
        return ol + 2

p, score, positions, velocities = pso_solve(20, 10
                                            , init_positions
                                            , init_velocities
                                            , fitness
                                            , lambda p: choice.apply_constraints(p[0][0], p[0][1])
                                            , lambda score: score < 0.8
                                            , False
                                            )

print(p)
print(score)

d:\SVD\tesis\logChoice.py:11: RuntimeWarning: overflow encountered in scalar power
  exp = x ** (1 / b)


[[ 0.17213467 -0.18865357]]
1.7919999999999998


In [13]:
def test(choice, p):
    choiceParameters = FixedParameters()
    problem = MemoryGame(choice, choiceParameters, 0.03, 36, 1000)
    total = 0
    count = 100
    ol = 0
    choiceParameters.set(p[0][0], p[0][1])

    for i in range(count):
        cards = problem.random_cards()
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        print(total / (count - ol), ol)
    else:
        print(ol)

In [14]:
test(LogChoice(), p)

87.14 0


In [21]:
test(BigChoice(), p)

29.858823529411765 15


In [ ]:
choice = LogChoice()
choiceParameters = FixedParameters()
problem = MinimumVertexCoverVertices(choice, choiceParameters)

In [25]:
dataset = []

In [26]:
for i in range(300):
    n = random.randint(1, 100)
    graph = generate_random_graph(n)
    m = smallest_dominating_set(graph) 
    n = greedy_dominating_set(graph)

    if n > m:
        dataset.append((graph, n))

print(len(dataset))

78


In [27]:
init_positions = lambda num_particles: [np.random.random((1, 2)) * 2 - 1 for i in range(num_particles)]
init_velocities = lambda num_particles: [np.zeros((1, 2)) for i in range(num_particles)]   

def fitness(p):
    losses = 0
    choiceParameters.set(p[0][0], p[0][1])

    for graph, greedy in dataset:
        if problem.run(graph) >= greedy:
            losses += 1

    return losses

p, score, positions, velocities = pso_solve(20, 10
                                            , init_positions
                                            , init_velocities
                                            , fitness
                                            , lambda p: choice.apply_constraints(p[0][0], p[0][1])
                                            , lambda score: score == 0
                                            , False
                                            )

print(p)
print(score)

[[ 1.00000000e-11 -5.81720977e-01]]
63


In [28]:
count = 200
total = 0

choiceParameters.set(p[0][0], p[0][1])

for j in range(count):
    win_C = 0

    for i in range(count):
        n = random.randint(1, 200)
        graph = generate_random_graph(n)
        m = problem.run(graph) 
        n = greedy_dominating_set(graph)

        if min(m, n) < n:
            win_C += 1

    total += win_C 

print(f"won: {total/count}")

won: 8.355
